# Getting started

In [3]:
from fasthtml import FastHTML
from fasthtml.common import *
from starlette.testclient import TestClient
from pprint import pprint

app = FastHTML()

@app.get("/")
def home():
    return "<h1>Hello, World</h1>" # text with html tags


hxhdr = {'headers':{'hx-request':"1"}} # mock to remove the headers
client = TestClient(app)
pprint(client.get("/", **hxhdr).text)


# serve()

'<h1>Hello, World</h1>'


# Create a html page

In [7]:
from fasthtml.common import *

page = Html(
    Head(Title('Some page')),
    Body(
        Div('Some text, ', 
            A('A link', href='https://example.com'), 
            Img(src="https://placehold.co/200"), 
            cls='myclass')))

pprint(page) # page: is about FT
print("====================")
print(to_xml(page)) # xml format
print("====================")
show(page) # actual html page

['html',
 (['head', (['title', ('Some page',), {}],), {}],
  ['body',
   (['div',
     ('Some text, ',
      ['a', ('A link',), {'href': 'https://example.com'}],
      ['img', (), {'src': 'https://placehold.co/200'}]),
     {'class': 'myclass'}],),
   {}]),
 {}]
<html>
  <head>
    <title>Some page</title>
  </head>
  <body>
    <div class="myclass">
Some text, 
      <a href="https://example.com">A link</a>
      <img src="https://placehold.co/200">
    </div>
  </body>
</html>



In [8]:
app = FastHTML()

@app.get("/")
def home():
    return Div(
                H1('Hello, World'), 
                P('Some text'), 
                P('Some more text'))


hxhdr = {'headers':{'hx-request':"1"}} # mock to remove the headers
client = TestClient(app)
pprint(client.get("/", **hxhdr).text)

('<div>\n'
 '  <h1>Hello, World</h1>\n'
 '  <p>Some text</p>\n'
 '  <p>Some more text</p>\n'
 '</div>\n')


# How to style

In [9]:
from fasthtml.common import *

css = Style(':root { --pico-font-size: 100%; --pico-font-family: Pacifico, cursive;}')
app = FastHTML(hdrs=(picolink, css)) # custom styling to override the pico defaults

@app.route("/")
def get():
    op1 = Title("Hello World"), Main(H1('Hello, World'), cls="container") 
    op2  = Titled('Hello World') #exactly the same as above, and no need to worry about cls="container"
    return op2

# hxhdr = {'headers':{'hx-request':"1"}} # mock to remove the headers
# client = TestClient(app)
# pprint(client.get("/", **hxhdr).text)

serve()

('<title>Hello World</title>\n'
 '\n'
 '<main class="container">\n'
 '  <h1>Hello, World</h1>\n'
 '</main>\n')


### `Main` and `cls="container"`

In HTML, the `<main>` tag is used to wrap the main content of the body of a document or an application. The content inside the `<main>` tag should be unique to the document, excluding content that is repeated across a set of documents such as site navigation links, header or footer information.

When the statement says "we put all of our content inside a `<main>` tag with a class of `container`", it means that all the primary content of the webpage is wrapped inside a `<main>` tag. The `class="container"` attribute is used to apply specific CSS styling to this `<main>` element.

In many CSS frameworks like Bootstrap or PicoCSS, the `container` class is used to center the content and handle the layout in a certain way, often providing a responsive design. The exact styling applied by the `container` class can vary depending on the CSS rules defined in the linked stylesheets.

In the provided Python code, `Main(H1('Hello, World'), cls="container")` is creating a `<main>` HTML element with the class `container`, and inside this `<main>` element, it's placing an `<h1>` element with the text 'Hello, World'.

# Form send data without htmx

## use`action='/'`, `method='POST'` for Form


# How to multi-page app

In [ ]:
from fasthtml.common import *

app = FastHTML()
messages = ["This is a message, which will get rendered as a paragraph"]

@app.get("/")
def home():
    return Main(H1('Messages'), 
                *[P(msg) for msg in messages], # spread the list of messages as P elements
                A("Link to Page 2 (to add messages)", 
                  href="/page2")) # go to a new page

@app.get("/page2")
def page2():
    return Main(P("Add a message with the form below:"),
                Form(Input(type="text", 
                           name="data"), # create a (name:value) pair, which is "data":<input>
                     Button("Submit"),
                     action="/", method="post"))

@app.post("/")
def add_message(data:str): # here we can access the data from the form
    messages.append(data)
    return home()

serve()


# `hx_target='#count'` is the same  `target_id='count'`

# `global`: how use `count` across all pages

In [ ]:
from fasthtml.common import *

app = FastHTML()

count = 0

@app.get("/")
def home():
    return Title("Count Demo"), Main(
        H1("Count Demo"),
        P(f"Count is set to {count}", id="count"),
        Button("Increment", 
            #    hx_get="/increment", 
               hx_post="/increment",
            #    hx_target="#count", 
               target_id="count",
               hx_swap="innerHTML")
    )

# @app.get("/increment")
@app.post("/increment")
def increment():
    print("incrementing") # debug print into the terminal
    global count # access the global variable count
    count += 1
    return f"Count is set to {count}" # return it as the content of the P with id "count"

serve()